### Cálculo del NVDI para los proyectos visitados

### Cargue de las zonas de proyectos visitados y de un buffer de 10 KM para cada punto

In [452]:

import sys
from mapas_raster import *
import ee
import geopandas as gpd
import requests
import zipfile

from osgeo import ogr
import json

import config
import urllib.request
import rasterio
import numpy as np
from xml.dom import minidom
import matplotlib.pyplot as plt
import imageio
from rasterio.mask import mask
from rasterio.features import geometry_mask
from shapely.geometry import mapping, shape
from rasterio.warp import transform_geom

In [20]:
import os
from dotenv import load_dotenv
import planet
from planet import Auth
import requests
import pandas as pd

In [12]:
#Cargar credenciales
load_dotenv()

PLANET_API_KEY = os.getenv("PLANET_API_KEY")
PLANET_USER=os.getenv("PLANET_USER")
PLANET_PASS=os.getenv("PLANET_PASS")

In [18]:
#Folder de datos
data_folder = '/Users/Daniel/Library/CloudStorage/OneDrive-VestigiumMétodosMixtosAplicadosSAS/geoinfo/Colombia/Bogotá/bosques_bogota'

### Cálculo de NVDI
El cálculo del NVDI lo hacemos con base en este Tutorial de Planet:
- https://developers.planet.com/docs/planetschool/calculate-an-ndvi-in-python/?utm_source=google&utm_medium=paid-search&gad=1&gclid=CjwKCAjw6p-oBhAYEiwAgg2Pgnhzy0eAFD2MZSrSbYX7Q_i_ZDXBIL5AomP3D60wIud-_EHnZri4UxoCY80QAvD_BwE

In [13]:
#Autenticación
planet.Auth.from_login(PLANET_USER, PLANET_PASS)

In [14]:
API_KEY = PLANET_API_KEY

In [17]:
#setup Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"

#setup session
session = requests.Session()

#authenticate
session.auth = (API_KEY, "")

In [21]:
#Select mosaic
mosaicos_path = os.path.join(data_folder, 'planet/planet_mosaics.xlsx')
#Import Excel with mosaic names
df2 = pd.read_excel(mosaicos_path)

### Descarga de las imágenes de Planet

In [46]:
def get_mosaic_id(mosaic_name):
    #set params for search using name of mosaic
    parameters = {
        "name__is" :mosaic_name
    }

    #make get request to access mosaic from basemaps API
    res = session.get(API_URL, params = parameters)
    
    #print metadata for mosaic
    mosaic = res.json()
    
    #get id
    mosaic_id = mosaic['mosaics'][0]['id']
    #get bbox for entire mosaic
    mosaic_bbox = mosaic['mosaics'][0]['bbox']
    #Get the mosaics' CRS
    mosaic_crs = mosaic['mosaics'][0]['coordinate_system']
    #converting bbox to string for search params
    string_bbox = ','.join(map(str, mosaic_bbox))
    
    return mosaic_id

In [47]:
def extract_after_word(text, word, x):
    """
    Extracts x number of characters after a specified word in the given text.
    
    Parameters:
    - text (str): The input string from which characters need to be extracted.
    - word (str): The word after which characters will be extracted.
    - x (int): Number of characters to be extracted after the word.
    
    Returns:
    - str: Extracted characters. If the word isn't found or there aren't enough characters after the word, an appropriate message is returned.
    """
    
    try:
        position = text.index(word) + len(word)
        return text[position: position + x].strip()
    except ValueError:
        return "The word '{}' was not found in the given text.".format(word)
    except:
        return "There aren't enough characters after the word '{}'.".format(word)

In [48]:
gdf_row = gdf.loc[[0]]

In [49]:
gdf_row.to_clipboard()

In [58]:
def download_planet_images(mosaic_name, polygon, save_dir, file_name, original_crs, inicio_fin=None): 
    
    #search for mosaic quad using AOI
    bbox = get_bounding_box_string(polygon, original_crs, 'WGS84')
    
    search_parameters = {
        'bbox': bbox,
        'minimal': True
    }
    
    mosaic_id = get_mosaic_id(mosaic_name)

    #accessing quads using metadata from mosaic
    quads_url = "{}/{}/quads".format(API_URL, mosaic_id)

    #send request
    res = session.get(quads_url, params=search_parameters, stream=True)
    quads = res.json()
    items = quads['items']
    
    #Guardar el archivo
    link = items[0]['_links']['download']
    
    #Crear el nombre del archivo
    name = file_name + '_' + extract_after_word(mosaic_name, 'analytic_', 7) + '_'
    
    if inicio_fin=='inicio': 
        name = name + 'inicio.tiff'
    
    if inicio_fin=='fin':
        name = name + 'fin.tiff'
    
    if inicio_fin==None: 
        name = name + '.tiff'

    DIR = os.path.join(save_dir, file_name) 
    os.makedirs(DIR, exist_ok=True)
    
    filename = os.path.join(DIR, name)
    urllib.request.urlretrieve(link, filename)
    
    return filename

In [59]:
gdf_crs = gdf.crs

In [121]:
#Descargar todas las imágenes de fin y guardar las rutas
gdf['archivo_fin'] = gdf.head(1).apply(lambda row: download_planet_images(row['closest_inicio'], 
                                             row['geometry'], 
                                             "MMC - General - FIDA/ndvi", 
                                             row['id'], 
                                             original_crs=gdf_crs,
                                             inicio_fin='inicio'), 
                                             axis=1)

0    MMC - General - FIDA/ndvi/NICADAPTA_1/NICADAPT...
dtype: object

In [130]:
gdf.head(3).apply(lambda row: download_planet_images(row['closest_inicio'], 
                                             row['geometry'], 
                                             "MMC - General - FIDA/ndvi", 
                                             row['id'], 
                                             original_crs=gdf_crs,
                                             inicio_fin='inicio'), 
                                             axis=1)

0    MMC - General - FIDA/ndvi/NICADAPTA_1/NICADAPT...
1    MMC - General - FIDA/ndvi/NICAVIDA_1/NICAVIDA_...
2    MMC - General - FIDA/ndvi/NICAVIDA_2/NICAVIDA_...
dtype: object

## Crop images 

In [136]:
def crop_rasters_to_gdf(gdf, raster_path_col, output_dir, cropped_raster_path):
    """
    Function to crop rasters based on bounding box of geometries in GeoDataFrame.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    def crop_raster(row):
        raster_path = row[raster_path_col]
        with rasterio.open(raster_path) as src:

            # Ensure that the raster and geometry have the same CRS
            geom = row['geometry']
            if gdf.crs != src.crs:
                geom = gpd.GeoSeries([geom], crs=gdf.crs).to_crs(src.crs).iloc[0]
            
            # Check for intersection between raster bounds and geometry bounds
            left, bottom, right, top = src.bounds
            raster_bbox = box(left, bottom, right, top)
            if not raster_bbox.intersects(geom):
                print(f"Warning: Geometry does not overlap with raster {raster_path}")
                return None
            
            # Crop the raster
            out_image, out_transform = mask(src, [geom], crop=True)
            out_meta = src.meta.copy()
            
            # Update metadata
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})

            # Construct output raster path
            base_name = os.path.basename(raster_path)
            cropped_path = os.path.join(output_dir, f"cropped_{base_name}")
            
            # Write the cropped raster to disk
            with rasterio.open(cropped_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            return cropped_path

    # Apply the crop function to each row in the GeoDataFrame
    gdf[cropped_raster_path] = gdf.apply(crop_raster, axis=1)

    return gdf

In [140]:
gdf = crop_rasters_to_gdf(gdf, 'archivo_inicio', 'MMC - General - FIDA/ndvi/crooped', 'archivo_inicio_cropped')

## Creación de imágenes visibles en png

In [226]:
folder = '/Users/Daniel/Library/CloudStorage/OneDrive-SharedLibraries-VestigiumMétodosMixtosAplicadosSAS/MMC - General - FIDA/ndvi/cropped/'
os.chdir(folder)

In [227]:
def normalize_band(band, lower_percentile=2, upper_percentile=98):
    """
    Normalize a single band to a byte (0-255) scale using percentiles.
    """
    min_val = np.percentile(band, lower_percentile)
    max_val = np.percentile(band, upper_percentile)
    
    band_normalized = (band - min_val) / (max_val - min_val)
    band_normalized = np.clip(band_normalized, 0, 1)  # ensure all values are between 0 and 1
    return (band_normalized * 255).astype(np.uint8)


def tif_to_png(tif_path, png_path):
    """
    Converts a Planet .tif file to a visible PNG image.
    
    Parameters:
    - tif_path (str): Path to the input .tif file.
    - png_path (str): Path to save the output PNG image.
    """
    
    with rasterio.open(tif_path) as src:
        # Assuming the image is a standard 4-band PlanetScope image (BGRN)
        blue = normalize_band(src.read(1))
        green = normalize_band(src.read(2))
        red = normalize_band(src.read(3))
    
    # Stack RGB bands
    rgb = np.stack((red, green, blue), axis=-1)
    
    # Use imageio library to save the png
    import imageio
    imageio.imsave(png_path, rgb)
    
    #print(f"Saved visible PNG image to {png_path}")

In [228]:
def get_files_in_folder(directory):
    """Return a list of file names in the given directory."""
    with os.scandir(directory) as entries:
        return [entry.name for entry in entries if entry.is_file()]

In [229]:
files = get_files_in_folder(folder)

In [230]:
for i in files: 
    tif_path = i
    if i.endswith('.tiff'):
        png_path = i.replace('.tiff', '.png')
        tif_to_png(tif_path, png_path)
    else: 
        continue

## Cálculo del NVDI

In [231]:
def calculate_ndvi(image_file, output_ndvi_file, cmap_nvdi_image):
    """
    Calculate the NDVI for a given image.
    
    Parameters:
    - image_file (str): Path to the image file.
    - output_ndvi_file (str): Path to save the output NDVI file. If the file already exists, it will be overwritten.
    - cmap_nvdi_image (str): Path to save the colormap NDVI image.
    
    Returns:
    None
    """

    # Load red and NIR bands - note all PlanetScope 4-band images have band order BGRN
    with rasterio.open(image_file) as src:
        band_red = src.read(3)
        band_nir = src.read(4)

    # Calculate NDVI while avoiding 0/0 situation
    denominator = band_nir + band_red + 1e-10
    ndvi = (band_nir.astype(float) - band_red.astype(float)) / denominator

    # Clip NDVI to the range [-1, 1]
    ndvi = np.clip(ndvi, -1, 1)

    # Convert NaN values to a chosen value (0 in this case)
    ndvi = np.nan_to_num(ndvi)

    # Normalize NDVI to [0, 1] for visualization
    ndvi_normalized = (ndvi + 1) / 2.0

    # Set spatial characteristics of the output object to mirror the input
    kwargs = src.meta
    kwargs.update(
        dtype=rasterio.float32,
        count = 1)

    # Check if the output raster already exists, if not, create it
    if os.path.exists(output_ndvi_file):
        os.remove(output_ndvi_file)

    # Create (or overwrite) the NDVI file
    with rasterio.open(output_ndvi_file, 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))
        
    # Save the normalized NDVI as a colormap image
    plt.imsave(cmap_nvdi_image, ndvi_normalized, cmap=plt.cm.RdYlGn)

In [232]:
for i in files: 
    
    if i.endswith('.tiff'):
        ndvi_path = i
        output_ndvi_file = i.replace('.tiff', '_ndvi.tiff')
        nvdi_png_image = i.replace('.tiff', '_ndvi.png')
        calculate_ndvi(ndvi_path, output_ndvi_file, nvdi_png_image)
    else: 
        continue

In [233]:
files = get_files_in_folder(folder)

In [294]:
def filter_by_substring(elements, target_string):
    """
    Filters elements of a list based on the presence of a target substring.
    
    Parameters:
    - elements (list): List of elements to be filtered.
    - target_string (str): Substring used for filtering.
    
    Returns:
    - list: Elements of the input list that contain the target substring.
    """
    return [element for element in elements if target_string in element]


def sort_by_endings(files, endings):
    """
    Sort the 'files' list based on the order of 'endings'.
    
    Parameters:
    - files (list): List of file names.
    - endings (list): List of endings to sort by.
    
    Returns:
    List: Sorted list of file names.
    """
    return sorted(files, key=lambda x: [x.endswith(ending) for ending in endings].index(True))

def get_image_files_project(project, files):
    files_1 = filter_by_substring(files, project)
    images = filter_by_substring(files_1, '.png')
    endings = ['fin_ndvi.png', 'fin.png', 'inicio_ndvi.png', 'inicio.png']
    
    return sort_by_endings(images, endings)

def get_titles_from_images(images_files, project):
    titles = []
    for i in images_files: 
        titles = titles + [extract_after_word(i, project+'_', 7)]
    return titles


def create_mosaic(png_files, titles, mosaic_title, note, output_file):
    """
    Create a 2x2 mosaic of 4 PNG images with titles and save as a new PNG.

    Parameters:
    - png_files (list): List of 4 paths to PNG files.
    - titles (list): List of 4 titles for each PNG.
    - mosaic_title (str): Title for the mosaic.
    - note (str): Note to be added at the end of the mosaic.
    - output_file (str): Path to save the output mosaic PNG.
    
    Returns:
    None
    """

    if len(png_files) != 4 or len(titles) != 4:
        raise ValueError("Please provide exactly 4 PNG files and 4 titles.")

    fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # Adjust size as needed

    for ax, png_file, title in zip(axs.ravel(), png_files, titles):
        img = plt.imread(png_file)
        ax.imshow(img)
        ax.set_title(title)
        ax.axis('off')

    plt.suptitle(mosaic_title)
    fig.subplots_adjust(bottom=0.15)
    fig.text(0.5, 0.08, note, ha='center')

    plt.savefig(output_file, dpi=300)  # Adjust dpi as needed
    plt.close()  # Close the plot to prevent it from being shown in the notebook


def get_mosaic_file(project, files, lista_proyectos):
    
    images = get_image_files_project(project, files)
    
    titles = get_titles_from_images(images, project)
    
    out_file_path = 'out/{}.png'.format(project)
    proyectos = ', '.join(map(str, lista_proyectos))
    
    create_mosaic(images, 
              titles, 
              'Análisis del punto ' + project, 
              'Proyectos que operaron en el municipo: ' + proyectos,
              out_file_path)
    

In [300]:
for index, row in gdf.iterrows():
    get_mosaic_file(row['id'], files, row['proyectos_activos'])

In [396]:
def get_nvdi_raster_files_project(files, project):
    files_1 = filter_by_substring(files, project)
    rasters = filter_by_substring(files_1, '.tiff')
    endings = ['inicio_ndvi.tiff', 'fin_ndvi.tiff', 'fin.tiff',  'inicio.tiff']
    rasters = sort_by_endings(rasters, endings)
    return rasters[0:2]

In [399]:
gdf['ndvi_paths'] = gdf.apply(lambda x: get_nvdi_raster_files_project(files, x['id']), axis=1)
gdf['ndvi_inicio_path'] = gdf['ndvi_paths'].apply(lambda x: x[0])
gdf['ndvi_final_path'] = gdf['ndvi_paths'].apply(lambda x: x[1])

In [403]:
def calculate_mean_ignoring_zeros(raster_path):
    """
    Calculate the mean value of a raster after setting all 0 values to NaN.

    Parameters:
    - raster_path: path to the raster file.

    Returns:
    - Mean value of the raster ignoring 0 values.
    """
    # Read the raster
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)  # Reading the first band
        
        # Convert zeros to NaN
        raster_data[raster_data == 0] = np.nan
        
        # Calculate the mean while ignoring NaN values
        mean_val = np.nanmean(raster_data)
    
    return mean_val


In [406]:
gdf['ndvi_inicio'] = gdf['ndvi_inicio_path'].apply(calculate_mean_ignoring_zeros)
gdf['ndvi_final'] = gdf['ndvi_final_path'].apply(calculate_mean_ignoring_zeros)

In [408]:
gdf.columns

Index(['geometry', 'proyecto_revisar', 'proyectos_activos', 'closest_inicio',
       'closest_fin', 'id', 'archivo_inicio', 'archivo_fin',
       'archivo_inicio_cropped', 'archivo_fin_cropped', 'nvdi_inicio',
       'nvdi_paths', 'ndvi_paths', 'ndvi_inicio', 'ndvi_final',
       'ndvi_inicio_path', 'ndvi_final_path'],
      dtype='object')

In [410]:
df1 = gdf[['proyecto_revisar', 'id', 'ndvi_inicio', 'ndvi_final']]

In [432]:
df1.loc[df1['ndvi_dif'].idxmax()]

proyecto_revisar      NICADAPTA
id                  NICADAPTA_3
ndvi_inicio            0.598055
ndvi_final             0.807385
ndvi_dif                0.20933
Name: 28, dtype: object

In [419]:
df1['ndvi_dif'] = df1['ndvi_final'] - df1['ndvi_inicio'] 

/var/folders/wn/vzyjwy2x00q4stf523pr30jw0000gn/T/ipykernel_94661/2452435540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ndvi_dif'] = df1['ndvi_final'] - df1['ndvi_inicio']


In [435]:
df1.sort_values(by='ndvi_dif')

,proyecto_revisar,id,ndvi_inicio,ndvi_final,ndvi_dif
22,Rural Adelante,Rural Adelante_6,0.810466,0.742784,-0.067681
26,Rural Adelante,Rural Adelante_8,0.780111,0.715836,-0.064275
27,Emprende Sur,Emprende Sur_3,0.741542,0.710910,-0.030633
4,NICAVIDA,NICAVIDA_4,0.779979,0.764730,-0.015249
20,Rural Adelante,Rural Adelante_5,0.750584,0.751463,0.000878
0,NICADAPTA,NICADAPTA_1,0.721407,0.728019,0.006612
16,Rural Adelante,Rural Adelante_3,0.607510,0.621182,0.013672
2,NICAVIDA,NICAVIDA_2,0.744284,0.758496,0.014213
3,NICAVIDA,NICAVIDA_3,0.744284,0.758496,0.014213
21,PROLENCA,PROLENCA_4,0.730585,0.745322,0.014737


In [423]:
df1[['proyecto_revisar', 'ndvi_inicio', 'ndvi_final', 'ndvi_dif']].groupby('proyecto_revisar').mean().to_clipboard()

In [436]:
gdf.head()

,geometry,proyecto_revisar,proyectos_activos,closest_inicio,closest_fin,id,archivo_inicio,archivo_fin,archivo_inicio_cropped,archivo_fin_cropped,nvdi_inicio,nvdi_paths,ndvi_paths,ndvi_inicio,ndvi_final,ndvi_inicio_path,ndvi_final_path
0,"POLYGON ((572966.330 1436176.904, 572961.515 1...",NICADAPTA,[NICADAPTA],planet_medres_normalized_analytic_2015-12_2016...,planet_medres_normalized_analytic_2020-12_mosaic,NICADAPTA_1,MMC - General - FIDA/ndvi/NICADAPTA_1/NICADAPT...,MMC - General - FIDA/ndvi/NICADAPTA_1/NICADAPT...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,"[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICADAPTA_1_2015-12_inicio_ndvi.tiff,...",0.721407,0.728019,cropped_NICADAPTA_1_2015-12_inicio_ndvi.tiff,cropped_NICADAPTA_1_2020-12_fin_ndvi.tiff
1,"POLYGON ((563423.637 1430336.447, 563418.822 1...",NICAVIDA,"[NICADAPTA, NICAVIDA]",planet_medres_normalized_analytic_2016-12_2017...,planet_medres_normalized_analytic_2022-12_mosaic,NICAVIDA_1,MMC - General - FIDA/ndvi/NICAVIDA_1/NICAVIDA_...,MMC - General - FIDA/ndvi/NICAVIDA_1/NICAVIDA_...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,"[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...",0.673739,0.759908,cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff,cropped_NICAVIDA_1_2022-12_fin_ndvi.tiff
2,"POLYGON ((570022.652 1434210.342, 570017.837 1...",NICAVIDA,"[NICADAPTA, NICAVIDA]",planet_medres_normalized_analytic_2016-12_2017...,planet_medres_normalized_analytic_2022-12_mosaic,NICAVIDA_2,MMC - General - FIDA/ndvi/NICAVIDA_2/NICAVIDA_...,MMC - General - FIDA/ndvi/NICAVIDA_2/NICAVIDA_...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,"[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_2_2016-12_inicio_ndvi.tiff, ...",0.744284,0.758496,cropped_NICAVIDA_2_2016-12_inicio_ndvi.tiff,cropped_NICAVIDA_2_2022-12_fin_ndvi.tiff
3,"POLYGON ((570022.652 1434210.342, 570017.837 1...",NICAVIDA,"[NICADAPTA, NICAVIDA]",planet_medres_normalized_analytic_2016-12_2017...,planet_medres_normalized_analytic_2022-12_mosaic,NICAVIDA_3,MMC - General - FIDA/ndvi/NICAVIDA_3/NICAVIDA_...,MMC - General - FIDA/ndvi/NICAVIDA_3/NICAVIDA_...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,"[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_3_2016-12_inicio_ndvi.tiff, ...",0.744284,0.758496,cropped_NICAVIDA_3_2016-12_inicio_ndvi.tiff,cropped_NICAVIDA_3_2022-12_fin_ndvi.tiff
4,"POLYGON ((551765.794 1467342.321, 551760.978 1...",NICAVIDA,"[NICADAPTA, NICAVIDA]",planet_medres_normalized_analytic_2016-12_2017...,planet_medres_normalized_analytic_2022-12_mosaic,NICAVIDA_4,MMC - General - FIDA/ndvi/NICAVIDA_4/NICAVIDA_...,MMC - General - FIDA/ndvi/NICAVIDA_4/NICAVIDA_...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,MMC - General - FIDA/ndvi/crooped/cropped_NICA...,"[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_1_2016-12_inicio_ndvi.tiff, ...","[cropped_NICAVIDA_4_2016-12_inicio_ndvi.tiff, ...",0.779979,0.764730,cropped_NICAVIDA_4_2016-12_inicio_ndvi.tiff,cropped_NICAVIDA_4_2022-12_fin_ndvi.tiff
